# Feature Engineering.

Idea principal: Debido a la naturaleza de la problemática de este proyecto no se “limpian” datos para facilitar el modelo, se manipulan para *dificultarlo* de forma controlada, es decir, inducir un escenario en el que los conjuntos de validación y prueba tienen datos no observados durante el entrenamiento.

## A. Transformación de datos crudos en variables útiles para el aprendizaje automático

En este proyecto, los datos no corresponden a observaciones tabulares tradicionales, sino a un grafo de conocimiento representado mediante tripletas (h, r, t), donde h y t son entidades y r es una relación. En este contexto, no existen variables numéricas continuas sobre las cuales aplicar técnicas clásicas de ingeniería de características como escalamiento, discretización, transformaciones matemáticas o codificaciones one-hot.

Sin embargo, el rol que tradicionalmente cumple la ingeniería de características es reformular los datos de entrada para que contengan la información relevante que permita abordar una problemática específica. Bajo esta interpretación, la etapa implementada en la libreta corresponde a una ingeniería de características estructural, donde los datos son transformados a nivel de grafo, no a nivel de atributo.

En particular, se realizan operaciones controladas de particionado inductivo del conjunto de tripletas con dos objetivos distintos, alineados con los trabajos base del proyecto:

* Escenario InGram: se generan conjuntos de validación y prueba que contienen relaciones completamente nuevas, ausentes del conjunto de entrenamiento. Esta transformación fuerza al modelo a aprender representaciones relacionales capaces de generalizar a relaciones no vistas.

* Escenario OOKB (Out-Of-Knowledge-Base): se construyen conjuntos de validación y prueba que incluyen entidades nuevas, no presentes durante el entrenamiento, mientras que el conjunto de relaciones se mantiene constante.

Estas operaciones no generan nuevas variables explícitas, pero modifican deliberadamente la distribución y composición del conjunto de datos, asegurando que la información disponible para el modelo sea consistente con un escenario inductivo. Por lo tanto, esta etapa cumple una función equivalente a la ingeniería de características clásica, ya que define qué información estructural está disponible para el aprendizaje y cuál debe ser inferida por el modelo. Todas las decisiones de particionado se justifican directamente por los supuestos experimentales de los papers de referencia.

# B. Selección y extracción de características

Las técnicas clásicas de selección y extracción de características, como umbral de varianza, correlación, pruebas estadísticas (chi-cuadrado, ANOVA) o métodos de reducción de dimensionalidad (PCA, FA), no son aplicables en este problema, ya que dichas técnicas requieren un espacio de características explícito y generalmente numérico.

En el caso de los grafos de conocimiento, las “características” no existen de forma directa, sino que emergen posteriormente como embeddings aprendidos por el modelo a partir de la estructura relacional del grafo. Por esta razón, cualquier intento de aplicar selección o extracción de características previa al entrenamiento sería conceptualmente incorrecto y metodológicamente inconsistente con el enfoque de knowledge graph embedding.

No obstante, de manera análoga a la selección de características, la libreta implementa mecanismos de control estructural para reducir sesgos y evitar fugas de información (information leakage), tales como:

* Exclusión estricta de relaciones nuevas del conjunto de entrenamiento (InGram).

* Exclusión estricta de entidades nuevas del conjunto de entrenamiento (OOKB).

* Separación explícita y reproducible de los conjuntos train / validation / test.

Estas decisiones cumplen un rol equivalente al filtrado de características, ya que limitan la información disponible durante el entrenamiento, reduciendo la posibilidad de que el modelo aprenda atajos triviales y asegurando que la complejidad del aprendizaje esté alineada con el problema inductivo planteado. La “extracción” de representaciones queda delegada correctamente a la etapa de modelado, donde los embeddings son aprendidos de forma automática.

## C. Conclusiones de la fase de Preparación de los Datos (CRISP-ML)

La fase de preparación de los datos en este proyecto se adapta a la naturaleza relacional del problema y al objetivo inductivo planteado por los trabajos de referencia. Aunque no se aplican técnicas tradicionales de limpieza, transformación o selección de características, se ejecuta una preparación estructural deliberada del grafo, que resulta crítica para la validez experimental del modelo.

La construcción de escenarios InGram y OOKB mediante particiones controladas garantiza que el modelo sea evaluado bajo condiciones realistas de generalización, evitando fugas de información y sesgos estructurales. En este sentido, la preparación de los datos no busca simplificar el problema, sino definir explícitamente las limitaciones de conocimiento bajo las cuales el modelo debe operar, es decir, información no presente durante el entrenamiento.

Como resultado, esta etapa sienta las bases para un proceso de modelado sólido, reproducible y conceptualmente alineado con el objetivo del proyecto, asegurando que el desempeño observado refleje verdaderamente la capacidad inductiva del modelo.

# 1. Environment and GPU sanity check

In [1]:
# GPU check for future pipeline
import torch

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)

if not torch.cuda.is_available():
  print("------------No GPU. Set Runtime → Change runtime type → GPU------------")

try:
    import torch_geometric
    print("Torch Geometric:", torch_geometric.__version__)
except ModuleNotFoundError:
    print("Torch Geometric not found. Installing")
    torch_version = torch.__version__.split("+")[0]
    cuda_version = torch.version.cuda.replace(".", "")

    !python -m pip install -q pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv \
        -f https://data.pyg.org/whl/torch-{torch_version}+cu{cuda_version}.html

    !python -m pip install -q torch-geometric

Torch: 2.9.0+cu126
CUDA available: True
CUDA Version: 12.6
Torch Geometric not found. Installing
ERROR: Could not find a version that satisfies the requirement pyg-lib (from versions: none)
ERROR: No matching distribution found for pyg-lib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00


# 2. Dataset download and normalization

In [2]:
#!rm -r raw_data data

In [3]:
# =========================
# Dataset download & normalization
# =========================

from pathlib import Path
import requests
import pandas as pd
from torch_geometric.datasets import WordNet18RR, FB15k_237

# -------------------------
# Paths
# -------------------------
RAW_DIR  = Path("./raw_data")      # Raw / potentially dirty datasets
DATA_DIR = Path("./data/newlinks") # Normalized datasets (h, r, t) for New Links
OOKB_DIR = Path("./data/newentities") # Normalized datasets (h, r, t) for New Entities

OOKB_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(exist_ok=True)
DATA_DIR.mkdir(parents=True,exist_ok=True)

print(f"RAW_DIR : {RAW_DIR.resolve()}")
print(f"DATA_DIR: {DATA_DIR.resolve()}")

# -------------------------
# Helpers
# -------------------------
def normalize_to_txt(src_path: Path, dst_path: Path):
    """
    Read raw KG triple file src_path and saves first 3 columns
    as head<TAB>rel<TAB>tail into dst_path.
    """
    df = pd.read_csv(
        src_path,
        sep=None,
        engine="python",
        header=None,
        on_bad_lines="skip"
    )

    if df.shape[1] < 3:
        raise ValueError(
            f"[FORMAT ERROR] Invalid KG triple file: {src_path}\n"
            f"Detected columns: {df.shape[1]}\n"
            "Expected format: head, relation, tail, [optional extra columns]"
        )

    df.iloc[:, :3].to_csv(dst_path, sep="\t", index=False, header=False)


def pyg_dataset_to_standard(pyg_dataset, name: str):
    """
    Normalize (tab) PyG raw files from raw
    and saves as data/name/{train,valid,test}.txt
    into data/name
    """
    raw_dir = Path(pyg_dataset.raw_dir)
    out_dir = DATA_DIR / name
    out_dir.mkdir(exist_ok=True)

    print(f"\nProcessing PyG dataset: {name}")

    file_map = {
        "train": ["train.txt"],
        "valid": ["valid.txt", "valid.csv"],
        "test":  ["test.txt"]
    }

    for split, candidates in file_map.items():
        for fname in candidates:
            src = raw_dir / fname
            if src.exists():
                dst = out_dir / f"{split}.txt"
                normalize_to_txt(src, dst)
                print(f"  -> {split}.txt")
                break
        else:
            print(f"  [!] Missing split: {split}")


def download_file(url: str, dst: Path):
    if dst.exists():
        return
    print(f"Downloading {dst.name}...")
    r = requests.get(url)
    r.raise_for_status()
    dst.write_bytes(r.content)

# -------------------------
# PyG datasets
# -------------------------
print("\n--- Downloading PyG datasets ---")

wn18rr = WordNet18RR(root=RAW_DIR / "WordNet18RR")
pyg_dataset_to_standard(wn18rr, "WN18RR")

fb237 = FB15k_237(root=RAW_DIR / "FB15k-237")
pyg_dataset_to_standard(fb237, "FB15k-237")

# -------------------------
# External datasets
# -------------------------
print("\n--- Downloading external datasets ---")

EXTERNAL_DATASETS = {
    "CoDEx-M": "https://raw.githubusercontent.com/tsafavi/codex/master/data/triples/codex-m/",
    "WN11":    "https://raw.githubusercontent.com/KGCompletion/TransL/master/WN11/",
    "FB13":    "https://raw.githubusercontent.com/KGCompletion/TransL/master/FB13/",
}

for name, base_url in EXTERNAL_DATASETS.items():
    raw_out = RAW_DIR / name
    data_out = DATA_DIR / name
    raw_out.mkdir(exist_ok=True)
    data_out.mkdir(exist_ok=True)

    print(f"\n{name}")
    for split in ["train", "valid", "test"]:
        url = f"{base_url}{split}.txt"
        raw_path = raw_out / f"{split}.txt"
        data_path = data_out / f"{split}.txt"

        download_file(url, raw_path)
        normalize_to_txt(raw_path, data_path)
        print(f"  -> {split}.txt")

    if name != "CoDEx-M":
      for split in ["entity2id", "relation2id"]:
          url = f"{base_url}{split}.txt"
          raw_path = raw_out / f"{split}.txt"
          download_file(url, raw_path)

print("\n[DONE] All datasets downloaded and normalized.")


RAW_DIR : /content/raw_data
DATA_DIR: /content/data/newlinks

--- Downloading PyG datasets ---


Processing...
Done!



Processing PyG dataset: WN18RR
  -> train.txt
  -> valid.txt
  -> test.txt


Processing...
Done!



Processing PyG dataset: FB15k-237
  -> train.txt
  -> valid.txt
  -> test.txt

--- Downloading external datasets ---

CoDEx-M
  -> train.txt
  -> valid.txt
  -> test.txt

WN11
  -> train.txt
  -> valid.txt
  -> test.txt

FB13
  -> train.txt
  -> valid.txt
  -> test.txt

[DONE] All datasets downloaded and normalized.


# 3. New Link splits (InGram setup)

In [4]:
# =========================
# Inductive relation-based splits (NL-*)
# =========================

from pathlib import Path
import random
from collections import defaultdict

# -------------------------
# Config
# -------------------------
SEED = 42

# Defined as in paper, scenarios for different persentages of
# dataset links used as unseen in training
ALPHAS = {
    "NL-25": 0.25,
    "NL-50": 0.50,
    "NL-75": 0.75,
    "NL-100": 1.00,
}

# Reproducibility
random.seed(SEED)

# -------------------------
# IO helpers
# -------------------------
def read_triples(path: Path):
    triples = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            h, r, t = line.rstrip("\n").split("\t")
            triples.append((h, r, t))
    return triples


def write_triples(path: Path, triples):
    with path.open("w", encoding="utf-8") as f:
        for h, r, t in triples:
            f.write(f"{h}\t{r}\t{t}\n")


# -------------------------
# Core logic
# -------------------------
def generate_inductive_splits(dataset_dir: Path):
    """
    Generate inductive relation-based splits (NL-*) for a dataset directory.

    The input directory must contain:
        train.txt
        valid.txt
        test.txt

    The function creates, inside the same directory:
        NL-25/, NL-50/, NL-75/, NL-100/
    each containing train/valid/test splits where relations in valid/test
    are completely unseen during training.

    Parameters
    ----------
    dataset_dir : Path
        Path to a dataset directory under BASE_DATA_DIR.
    """
    train_path = dataset_dir / "train.txt"
    valid_path = dataset_dir / "valid.txt"
    test_path  = dataset_dir / "test.txt"

    if not (train_path.exists() and valid_path.exists() and test_path.exists()):
        print(f"[SKIP] {dataset_dir.name}: missing train/valid/test files")
        return

    print(f"\n[DATASET] {dataset_dir.name}")

    train = read_triples(train_path)
    valid = read_triples(valid_path)
    test  = read_triples(test_path)

    # All triples of dataset
    all_triples = train + valid + test

    # Group triples by relation
    rel2triples = defaultdict(list)
    for h, r, t in all_triples:
        rel2triples[r].append((h, r, t))

    # Relations
    relations = list(rel2triples.keys())
    num_relations = len(relations)

    print(f"  Total relations : {num_relations}")
    print(f"  Total triples   : {len(all_triples)}")

    for split_name, alpha in ALPHAS.items():
        # New = unseen at training triples
        # number of new triples
        n_new = int(round(num_relations * alpha))

        # Randomly selected
        shuffled = relations[:]
        random.shuffle(shuffled)

        # new links -> val/test
        # old links -> train
        new_rels = set(shuffled[:n_new])
        old_rels = set(shuffled[n_new:])

        # old links -> train
        train_split = []
        for r in old_rels:
            train_split.extend(rel2triples[r])

        # new links -> val/test
        new_triples = []
        for r in new_rels:
            new_triples.extend(rel2triples[r])

        # val/test -> 50%/50% of new links total
        random.shuffle(new_triples)
        mid = len(new_triples) // 2
        valid_split = new_triples[:mid]
        test_split  = new_triples[mid:]

        # Safety checks
        assert {r for _, r, _ in train_split}.isdisjoint(new_rels)
        assert {r for _, r, _ in valid_split}.issubset(new_rels)
        assert {r for _, r, _ in test_split}.issubset(new_rels)

        out_dir = dataset_dir / split_name
        out_dir.mkdir(exist_ok=True)

        write_triples(out_dir / "train.txt", train_split)
        write_triples(out_dir / "valid.txt", valid_split)
        write_triples(out_dir / "test.txt",  test_split)

        print(
            f"  [{split_name}] "
            f"new_rel={len(new_rels)} | "
            f"train={len(train_split)} | "
            f"valid={len(valid_split)} | "
            f"test={len(test_split)}"
        )



In [5]:
# -------------------------
# Run for all datasets
# -------------------------
print("\n=== Generating inductive splits for all datasets ===")

for dataset_dir in DATA_DIR.iterdir():
    if dataset_dir.is_dir():
        generate_inductive_splits(dataset_dir)

print("\n[DONE] All NL-* splits generated.")


=== Generating inductive splits for all datasets ===

[DATASET] CoDEx-M
  Total relations : 51
  Total triples   : 206205
  [NL-25] new_rel=13 | train=173839 | valid=16183 | test=16183
  [NL-50] new_rel=26 | train=137278 | valid=34463 | test=34464
  [NL-75] new_rel=38 | train=97895 | valid=54155 | test=54155
  [NL-100] new_rel=51 | train=0 | valid=103102 | test=103103

[DATASET] FB15k-237
  Total relations : 237
  Total triples   : 310116
  [NL-25] new_rel=59 | train=220712 | valid=44702 | test=44702
  [NL-50] new_rel=118 | train=126564 | valid=91776 | test=91776
  [NL-75] new_rel=178 | train=75589 | valid=117263 | test=117264
  [NL-100] new_rel=237 | train=0 | valid=155058 | test=155058

[DATASET] FB13
  Total relations : 13
  Total triples   : 375514
  [NL-25] new_rel=3 | train=317509 | valid=29002 | test=29003
  [NL-50] new_rel=6 | train=159199 | valid=108157 | test=108158
  [NL-75] new_rel=10 | train=135632 | valid=119941 | test=119941
  [NL-100] new_rel=13 | train=0 | valid=18775

# 4 New Entities splits (OOKB setup)

In [6]:
# -------------------------
# Config
# -------------------------
UNSEEN_RATIO = 0.20   # % de entidades OOKB

# -------------------------
# OOKB logic
# -------------------------
def generate_ookb_splits(dataset_dir: Path):
    train_path = dataset_dir / "train.txt"
    valid_path = dataset_dir / "valid.txt"
    test_path  = dataset_dir / "test.txt"

    if not (train_path.exists() and valid_path.exists() and test_path.exists()):
        print(f"[SKIP] {dataset_dir.name}: missing train/valid/test")
        return

    print(f"\n[OOKB DATASET] {dataset_dir.name}")

    train = read_triples(train_path)
    valid = read_triples(valid_path)
    test  = read_triples(test_path)

    # All triples of dataset
    all_triples = train + valid + test

    # Collect entities & relations
    entities = set()
    relations = set()
    for h, r, t in all_triples:
        entities.update([h, t])
        relations.add(r)

    entities  = list(entities)
    relations = list(relations)

    # Select unseen entities
    random.shuffle(entities)
    # Ratio-based selection
    n_unseen = int(round(len(entities) * UNSEEN_RATIO))
    unseen_entities = set(entities[:n_unseen])

    print(f"  entities={len(entities)} | unseen={len(unseen_entities)}")

    # Split triples
    train_split = []
    new_triples = []

    # new triples = triples not seen at training
    # train set directly assigned
    for h, r, t in all_triples:
        if h in unseen_entities or t in unseen_entities:
            new_triples.append((h, r, t))
        else:
            train_split.append((h, r, t))

    random.shuffle(new_triples)
    mid = len(new_triples) // 2

    # val/test -> 50%/50% of new links total
    valid_split = new_triples[:mid]
    test_split  = new_triples[mid:]

    # -------- safety checks --------
    assert all(
        h not in unseen_entities and t not in unseen_entities
        for h, _, t in train_split
    )

    assert any(
        h in unseen_entities or t in unseen_entities
        for h, _, t in valid_split + test_split
    )

    # -------- output --------
    out_dir = OOKB_DIR / dataset_dir.name
    out_dir.mkdir(exist_ok=True)

    write_triples(out_dir / "train.txt", train_split)
    write_triples(out_dir / "valid.txt", valid_split)
    write_triples(out_dir / "test.txt",  test_split)

    # In order to replicate OOKB structure
    # dictionaries need to be generated
    # -------- dictionaries --------
    entity2id = {e: i for i, e in enumerate(sorted(entities))}
    relation2id = {r: i for i, r in enumerate(sorted(relations))}
    unseenentity2id = {e: entity2id[e] for e in sorted(unseen_entities)}

    with (out_dir / "entity2id.txt").open("w") as f:
        for e, i in entity2id.items():
            f.write(f"{e}\t{i}\n")

    with (out_dir / "relation2id.txt").open("w") as f:
        for r, i in relation2id.items():
            f.write(f"{r}\t{i}\n")

    # Unseen dict is intended to be used as a reference
    # to perform semantic or text verification after predictions
    with (out_dir / "unseenentity2id.txt").open("w") as f:
        for e, i in unseenentity2id.items():
            f.write(f"{e}\t{i}\n")

    print(
        f"  train={len(train_split)} | "
        f"valid={len(valid_split)} | "
        f"test={len(test_split)}"
    )

In [7]:
# -------------------------
# Run OOKB for all datasets
# -------------------------

import shutil

OOKB_PREDEFINED = {"WN11", "FB13"}

# Predefined datasets already meet OOKB requirements
print("\n=== Preparing PREDEFINED OOKB datasets (from RAW) ===")

for name in OOKB_PREDEFINED:
    src = RAW_DIR / name
    dst = OOKB_DIR / name

    if not src.exists():
        print(f"[SKIP] {name}: not found in RAW_DIR")
        continue

    if dst.exists():
        print(f"[OK] {name}: already exists")
        continue

    shutil.copytree(src, dst)
    print(f"[COPIED] {name}")


# Custom datasets need to be prepared explicitly
print("\n=== Generating OOKB splits (custom datasets only) ===")

for dataset_dir in DATA_DIR.iterdir():
    if not dataset_dir.is_dir():
        continue

    if dataset_dir.name in OOKB_PREDEFINED:
        continue   # WN11 / FB13 ya están listos

    generate_ookb_splits(dataset_dir)

print("\n[DONE] All OOKB datasets generated.")



=== Preparing PREDEFINED OOKB datasets (from RAW) ===
[COPIED] FB13
[COPIED] WN11

=== Generating OOKB splits (custom datasets only) ===

[OOKB DATASET] CoDEx-M
  entities=17050 | unseen=3410
  train=138242 | valid=33981 | test=33982

[OOKB DATASET] FB15k-237
  entities=14541 | unseen=2908
  train=203159 | valid=53478 | test=53479

[OOKB DATASET] WN18RR
  entities=40943 | unseen=8189
  train=60305 | valid=16349 | test=16349

[DONE] All OOKB datasets generated.
